In [ ]:
import cv2, os
import numpy as np
from tqdm import tqdm
from PIL import Image
import open3d as o3d

In [ ]:
path = "EHF/"

data = []
for images in tqdm(os.listdir(path)):
    if images.endswith('.jpg'):
        image_path = path + images
        image = cv2.imread(image_path)
        image = cv2.resize(image,(256,256))
        data.append(image)

In [ ]:
def load_ply_data(ply_pat):
    pcd = o3d.io.read_point_cloud(ply_path)
    vertices = np.asarray(pcd.points)
    min_coords = np.min(vertices, axis=0)
    max_coords = np.max(vertices, axis=0)
    normalized_vertices = (vertices - min_coords) / (max_coords - min_coords)
    image_array = np.zeros((256, 256, 3), dtype=np.uint8)  # Adjust the image size as needed
    for vertex in normalized_vertices:
        x, y, z = vertex
        image_x = int(x * (image_array.shape[1] - 1))
        image_y = int((1 - y) * (image_array.shape[0] - 1))  # Reverse the y-coordinate
        image_array[image_y, image_x, :] = 255 
    image = Image.fromarray(image_array)
    image.save("check.jpg")
    img = cv2.imread("check.jpg")
    return img

In [ ]:
meshes = []

for images in tqdm(os.listdir(path)):
    if images.endswith('.ply'):
        ply_path = path + images
        ply_data = load_ply_data(ply_path)
        meshes.append(ply_data)

In [ ]:
data = np.array(data)#, dtype="float32") / 255.0
meshes = np.array(meshes)

In [ ]:
data.shape, meshes.shape

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, UpSampling2D, Concatenate, Reshape

In [ ]:
def unet(input_shape=(256,256,3)):
    inputs = Input(input_shape)
    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
    drop5 = Dropout(0.5)(conv5)

    # Decoder
    up6 = Conv2D(512, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(drop5))
    merge6 = Concatenate(axis=3)([drop4, up6])
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

    up7 = Conv2D(256, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv6))
    merge7 = Concatenate(axis=3)([conv3, up7])
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

    up8 = Conv2D(128, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv7))
    merge8 = Concatenate(axis=3)([conv2, up8])
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

    up9 = Conv2D(64, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv8))
    merge9 = Concatenate(axis=3)([conv1, up9])
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)
    conv9 = Conv2D(3, 1, activation='relu', padding='same')(conv9)

    model = Model(inputs=inputs, outputs=conv9)
    return model

In [ ]:
model = unet()
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(data, meshes, batch_size=4, epochs=10)

In [ ]:
model.save("3dcheck.h5")

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
image = cv2.imread("1.jpg")
image = cv2.resize(image,(256,256))
image1 = np.expand_dims(image,axis=0)
pred = model.predict(image1)[0]
cv2.imwrite("check.jpg", pred)